In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


In [2]:
#CONDITIONS
import re
import requests
from bs4 import BeautifulSoup
!pip install cloudscraper
import cloudscraper
import string
from tqdm import tqdm
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')


     |████████████████████████████████| 102kB 4.3MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [3]:
def get_corpus(paragraphs):
  """
  Input: list of strings
  Takes each string and strips it so that only nouns remain 
  Output: tuple, first item is a list of noun-only strings, 
    second is list of integers representing the length of the noun-only strings
  """
  nltk_types = ['NN', 'NNP', 'NNS', 'NNPS']  #nltk.help.upenn_tagset() to see docum.
  corpus = []
  lens = []
  for par in paragraphs:
    tokens = nltk.word_tokenize(par)
    tagged = nltk.pos_tag(tokens)
    nouns_only = ' '.join([word for word, typ in tagged if typ in nltk_types])
    corpus.append(nouns_only)
    lens.append(len(nouns_only.split()))
  lens = np.array(lens)[:, None]
  return corpus, lens


In [5]:
import pickle
import os.path
from os import path
from google.colab import drive

get_random_corpus = True
include_random_corpus = True

filepath = "/content/drive/MyDrive/"
#filepath = '/content/drive/MyDrive/MITDocuments/6.871 MLHC Project/'
corpus_pickle = filepath + 'corpus.pickle'

drive.mount('/content/drive')
with open(corpus_pickle, 'rb') as handle:
  corpus_conds, cond_lens, corpus_meds, med_lens, corpus_rands, rand_lens, corpus_conds_rand, corpus_meds_rand, vectorizer = pickle.load(handle)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
num_conds_total = len(corpus_conds)
num_meds_total = len(corpus_meds)
num_rands_total = len(corpus_rands)

vectorizer = CountVectorizer(ngram_range= (1, 1))
corpus_full_nums = vectorizer.fit_transform(corpus_conds + corpus_meds)

cnarr_conds = vectorizer.transform(corpus_conds).toarray()
cnarr_meds = vectorizer.transform(corpus_meds).toarray()
cnarr_rands = vectorizer.transform(corpus_rands).toarray()

# percent of articles with at least 1 occurence
word_freqs_conds = np.sum((cnarr_conds > 0).astype(int), axis = 0)/num_conds_total
word_freqs_meds = np.sum((cnarr_meds > 0).astype(int), axis = 0)/num_meds_total
word_freqs_rands = np.sum((cnarr_rands > 0).astype(int), axis = 0)/num_rands_total

# avg times per word per article
avg_word_counts_conds = np.sum(cnarr_conds[:(num_conds_total)]/cond_lens, axis = 0)/(num_conds_total)
avg_word_counts_meds = np.sum(cnarr_meds[:(num_meds_total)]/med_lens, axis = 0)/(num_meds_total)
avg_word_counts_rands = np.sum(cnarr_rands[:(num_rands_total)]/rand_lens, axis = 0)/(num_rands_total)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide


In [7]:
def get_google_def(med):
  #print("Searching google/webmd")
  drug_name = ""
  brand_names = []
  desc = "Not found" 
  ts = []
  ss = []
  i = 0
  url1 = "https://www.google.com/search?q=" + med + "+webmd"
  scraper = cloudscraper.create_scraper(delay = 15)
  my_text = scraper.get(url1).text
  soup = BeautifulSoup(my_text, "html.parser")
  
  links = soup.find_all('a')
  for l in links:
    try:
      if "https://www.webmd.com/drugs/2" in l.get('href'):
        k = l.get('href')
        myLink = k.split("details")[0] + "details"
        myLink = "https://" + k.split("https://")[1]
        #print(myLink)
        break
    except:
      continue

  if myLink is None:
    #print("No webmd link")
    return drug_name, brand_names, desc, ts, ss
  else:
    my_text = scraper.get(myLink).text
    soup = BeautifulSoup(my_text, "html.parser")
    try:
      desc = soup.find_all(class_="drug-uses")[-1].getText()
    except:
      desc = soup.find(class_="monograph-content").getText()
    #desc = desc.lower()
    #desc = desc.replace('\n', ' ')
    desc = desc.encode("ascii", "ignore")
    #print(med + " webmd: " + desc)

    try: 
      drug_name = soup.find(class_="drug-name").getText()
    except:
      drug_name = "Not found"
    
    try:
      bnames = soup.find_all(class_="drug-generic-name")
      for b in bnames:
        brand_names.append(b.getText())
    except:
      brand_names = brand_names
    return drug_name, brand_names, desc, ts, ss


def get_medline_description(med):
  drug_name = ""
  brand_names = []
  desc = "Not found" 
  ts = []
  ss = []
  URL_1 = 'https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus&v%3Asources=medlineplus-bundle&query=' + med + '&binning-state=group%3d%3dDrugs%20and%20Supplements&'
  scraper = cloudscraper.create_scraper(delay = 15)  
  my_text = scraper.get(URL_1).text
  soup = BeautifulSoup(my_text, "html.parser")

  #Going through the outputted urls in order, 
  #Picking the first one unless we deem it incorrect
  matches = soup.find_all(class_ = "url")
  names = soup.find_all(class_= "title")
  i = 0
  name = "Ravioli, ravioli, "
  URL_2 = "give me the formuoli"
  while (notCorrect(med, name, URL_2)):
    try:
      URL_2 = matches[i].text
    except:
      #print("No URL medline")
      desc = "Not found"
      return drug_name, brand_names, desc, ts, ss
    name = names[i].text
    i = i + 1
  # Going to the drug's page
  try:
    scraper = cloudscraper.create_scraper(delay = 15)
    my_text_2 = scraper.get(URL_2).text
    soup2 = BeautifulSoup(my_text_2, "html.parser")
    matches_2 = soup2.find_all(id = "why")
    desc = matches_2[0].contents[1].text
    #desc = desc.lower()
    #desc = desc.replace('\n', ' ')
    desc = desc.encode("ascii", "ignore")
    #print(med + ": " + desc)
    #Getting the drug name
    matches_3 = soup2.find(class_="with-also")
    drug_name = matches_3.text
    #Getting brand names for the drug
    try:
      matches_4 = soup2.find(id="section-brandname-1").contents[0]
    except:
      matches_4 = soup2.find(id="section-brand-name-2").contents[0]
    brand_names = []
    for li in matches_4.find_all('li'):
      brand_names.append(li.text)

    side_effect_matches = soup2.find(id = "side-effects")
    try:
      typical_sides = side_effect_matches.contents[1].contents[1]
      ts = []
      for li in typical_sides.find_all('li'):
        ts.append(li.text)
    except:
      ts = []
    
    try:
      severe_sides = side_effect_matches.contents[1].contents[3]
      ss = []
      for li in severe_sides.find_all('li'):
        ss.append(li.text)
      for li in severe_sides.parent.contents[2].find_all('li'):
        ss.append(li.text)
    except:
      ss = []
    return drug_name, brand_names, desc, ts, ss
  except:
    #print("Failed to read medline")
    desc = "Not found"
  return drug_name, brand_names, desc, ts, ss

#Helper function for get_med_descriptions
def get_med_description(med, get_side_effect = False):

  '''
  Search medlineplus for a description of med
  Input: string
  Output: string
  '''
  #print()
  med = str(med)
  med = med.lower()
  med = med.replace(" ", "+")

  drug_name = ""
  brand_names = []
  desc = "Not found" 
  ts = []
  ss = []

  try:
    drug_name, brand_names, desc, ts, ss = get_medline_description(med)
    if desc != "Not found" and med in desc.lower():
      return drug_name, brand_names, desc, ts, ss
  except:
    desc = "Not found"
  
  try:
    drug_name, brand_names, desc, ts, ss= get_google_def(med)
    if desc != "Not found":
      return drug_name, brand_names, desc, ts, ss
  except:
    #print("Failed")
    desc = "Not found"
  
  return drug_name, brand_names, desc, ts, ss


#Helper function for selecting valid drug search URL
#Add cases as we test
def notCorrect(med, name, URL):
  m = med.lower()
  n = name.lower()
  if name + URL == "Ravioli, ravioli, give me the formuoli":
    return True
  if "injection" not in m and "injection" in n:
    return True
  elif " and " not in m and " and " in n:
    return True
  elif "druginfo" not in URL:
    return True
  else:
    return False
  

In [9]:
def get_google_cond(cond):
  desc = "Not found"
  try:
    url1 = "https://www.google.com/search?q=" + cond + "+webmd"
    scraper = cloudscraper.create_scraper(delay = 15)
    my_text = scraper.get(url1).text
    soup = BeautifulSoup(my_text, "html.parser")
    
    links = soup.find_all('a')
    for l in links:
      try:
        if "https://www.webmd.com/" in l.get('href'):
          k = l.get('href')
          myLink = "https://" + k.split("https://")[1]
          #print(myLink)
          break
      except:
        continue
    
    if myLink is None:
      #print("No webmd link")
      return desc
    else:
      my_text = scraper.get(myLink).text
      soup = BeautifulSoup(my_text, "html.parser")
      try:
        desc = soup.find(class_="article-page active-page").getText()
      except:
        #print("Can't read webmd")
        return desc
      #desc = desc.lower()
      #desc = desc.replace('\n', ' ')
      desc = desc.encode("ascii", "ignore")
      #print(condition + ": " + desc)
    return desc
  except:
    return desc

def get_full_desc(condition):
  '''
  Search medlineplus for a description of condition
  Input: string
  Output: string
  '''
  try:
    cond = str(condition)
    cond = cond.replace(" ", "+")
    URL_1 = 'https://vsearch.nlm.nih.gov/vivisimo/cgi-bin/query-meta?v%3Aproject=medlineplus&v%3Asources=medlineplus-bundle&query=' + cond + '&binning-state=group%3d%3dHealth%20Topics&'
    scraper = cloudscraper.create_scraper(delay = 15)  
    my_text = scraper.get(URL_1).text
    soup = BeautifulSoup(my_text, "html.parser")

    #Going through the outputted urls in order, 
    #Picking the first one unless we deem it incorrect
    matches = soup.find_all(class_ = "url")
    names = soup.find_all(class_= "title")
    i = 0
    name = "Ravioli, ravioli, "
    URL_2 = "give me the formuoli"
    while (notCorrectCond(cond, name, URL_2)):
      try:
        URL_2 = matches[i].text
      except:
        desc = "Not found"
        return get_google_cond(cond)
      name = names[i].text
      i = i + 1

    scraper = cloudscraper.create_scraper(delay = 15)
    my_text_2 = scraper.get(URL_2).text
    soup2 = BeautifulSoup(my_text_2, "html.parser")

    #Getting the condition full description
    matches_2 = soup2.find_all(id = "topsum_section")
    matches_summary = matches_2[0]
    desc = matches_summary.getText()
    #desc = desc.lower()
    #desc = desc.replace('\n', ' ')
    desc = desc.encode("ascii", "ignore")
    #print(condition + ": " + desc)
    return desc
  except:
    desc = get_google_cond(cond)
    return desc


#Helper function for selecting valid drug search URL
#Add cases as we test
def notCorrectCond(cond, name, URL):
  m = cond.lower()
  n = name.lower()
  if name + URL == "Ravioli, ravioli, give me the formuoli":
    return True
  return False

In [10]:
def isImportantWord(i, condvec, medvec, condlen, medlen, word_list):
  """
  Function to determine if a word is "important". 
  - it must appear infrequently across all articles
    ie "disease" and "treatment" are too common but "cholesterol" and "antidepressant" are more unique
  - it must be more common for the given condition+medication than the average condition+medication
    ie "blood pressure" might appear once or twice in most articles, but 4 times for hypertension and hypertension drugs
  """

  cond_freq = .25
  med_freq = .25
  rand_freq =.01 #.0015 #we are actually referring to true rand freq/cond freq
  
  
  # important word if the word appears in less than 1/nth of the cond/med articles (+ wiki descriptions)
  if word_freqs_conds[i] < cond_freq and word_freqs_meds[i] < med_freq and word_freqs_rands[i]/word_freqs_conds[i] < rand_freq: 
    # important word if the word appears more in the chosen cond on average than it does in the average condition and likewise for med
    if condvec[0, i]/condlen > 1.5*avg_word_counts_conds[i] and medvec[0, i]/medlen > 1.5*avg_word_counts_meds[i]:
      #print(word_list[i] + " " +str(np.round(word_freqs_conds[i], 4)) + " " + str(np.round(word_freqs_meds[i], 4)) + " " + str(np.round(word_freqs_rands[i], 4)))
      return True
  return False


def get_mutual_words(cond, med, cond_descr, med_descr):
  """
  Finds (noun-only) words that are in both articles and are "important" (see helper function). 
  Selects the words that have highest frequency and outputs them as the keywords
  """

  # preprocess in the same way we did to the corpus
  cond_descr, clen = get_corpus([cond_descr])
  med_descr, mlen = get_corpus([med_descr])

  
  # cond_descr, med_descr, clen, mlen all are lists of len 1
  
  cond_vec = vectorizer.transform(cond_descr)
  med_vec = vectorizer.transform(med_descr)
  word_list = vectorizer.get_feature_names()
  
  # how many of each word in the paragraph (1d vectors)
  cond_word_count = cond_vec.toarray().sum(axis=0)
  med_word_count = med_vec.toarray().sum(axis=0)


  words_in_both_idx = np.logical_and(cond_word_count > 0, med_word_count > 0)
  both_idx = np.nonzero(words_in_both_idx)[0]
  mutual_words = []
  keywords = []
  for i in both_idx:
    if isImportantWord(i, cond_vec, med_vec, clen, mlen, word_list):
      mutual_words.append(word_list[i])
  if mutual_words:
    max_freq = sorted(mutual_words, key = lambda i: i[1], reverse = True)[0][1]
    keywords = [word for word in mutual_words if word[1]==max_freq]
  if (" " + cond.lower() + " ") in med_descr[0] and cond.lower() not in mutual_words:
    mutual_words.append(cond.lower())
    keywords.append(cond.lower())
  return keywords, mutual_words

def print_results(cond_med_pairs):
  had_results = 0
  for cond, med, cond_d, med_d in cond_med_pairs:
    keywords, results = get_mutual_words(cond, med, cond_d, med_d)
    print("cond: " + cond + ", med: " + med)
    print(f"KEYWORDS: {results} \n")
    had_results = had_results + 1 if results else had_results
  print(f"{had_results} had results of {len(cond_med_pairs)}, {100*had_results/len(cond_med_pairs)}%")

In [11]:
def describe(l):
  newL = []
  for el in l:
    cond, med = el
    cd = get_full_desc(cond)
    md = get_med_description(med)[2]
    try:
      cd = cd.decode('UTF-8')
    except:
      cd = cd
    try:
      md = md.decode('UTF-8')
    except:
      md = md
    newL.append((cond, med, cd, md))
  return newL


# Should have results
test = [
            ('diarrhea', 'lomotil'),
        ('hyperthyroidism', 'synthroid'),
        
]


print_results(describe(test))

cond: diarrhea, med: lomotil
KEYWORDS: ['diarrhea', 'fluids', 'infection'] 

cond: hyperthyroidism, med: synthroid
KEYWORDS: ['gland', 'goiters', 'hormone', 'hypothyroidism', 'thyroid'] 

2 had results of 2, 100.0%


In [12]:
# Should have results
related = [
            ('hypothyroidism', 'westhroid'),
            ('hypothyroidism', 'levoxyl'),
            ('hypothyroidism', 'cytomel'),
            ('hyperthyroidism', 'tapazole'),
            ('hyperthyroidism', 'synthroid'),
            ('hyperthyroidism', 'propylthiouracil'),
            ('hypotension', 'midodrine'),
            ('hypertension', 'lisinopril'),
            ('hypertension', 'sectral'),
            ('hypertension', 'atenolol'),
            ('hypertension', 'lotensin'),
            ('heart failure', 'lisinopril'),
            ('heart failure', 'furosemide'),
            ('heart failure', 'aldactone'),
            ('heart failure', 'valsartan'),
            ('CHF', 'carvedilol'),
            ('CHF', 'captopril'),
            ('CHF', 'losartan'),
            ('hypokalemia', 'aldactone'),
            ('hypokalemia', 'potassium chloride'), 
            ('hypokalemia', 'Klor-Con'),
            ('hyperkalemia', 'calcium gluconate'),
            ('hyperkalemia', 'insulin'),
            ('hyperkalemia', 'furosemide'),
            ('hyperlipidemia', 'atorvastatin'),
            ('hyperlipidemia', 'lovastatin'),
            ('hyperlipidemia', 'fenofibrate'),
            ('diabetes type 2', 'metformin'),
            ('diabetes type 2', 'insulin'),
            ('diabetes type 2', 'nateglinide'),
            ('diabetes type 2', 'glipizide'),  
            ('depression', 'wellbutrin'),
            ('depression', 'celexa'),
            ('depression', 'bupropion'),
            ('depression', 'clomipramine'),
            ('diarrhea', 'imodium'),
            ('diarrhea', 'lomotil'),
            ('diarrhea', 'loperamide'),
            ('cellulitis', 'cephalexin'),
            ('cellulitis', 'sulfamethoxazole'),
            ('cellulitis', 'clindamycin'),
            ('GERD', 'dexilant'),
            ('GERD', 'lansoprazole'),
            ('GERD', 'esomeprazole'),
            ('GERD', 'aciphex'),
            # added
            ('COPD', 'salbutamol'),
            ('COPD', 'terbutaline'),
            ('erectile dysfunction', 'tadalafil'),
            ('erectile dysfunction', 'viagra'),
            ('urinary tract infection', 'cephalexin'),
            ('UTI', 'fosfomycin'),
            ('cancer', 'abraxane'),
            ('cancer', 'doxil'),
            ('cancer', 'paraplatin'),
            ('anemia', 'integra'),
            ('anemia', 'folic acid'),
            ('anemia', 'procrit'),
            ('dementia', 'donepezil'),
            ('dementia', 'razadyne'),
            ('dementia', 'exelon'),
            
                     
]

print_results(describe(related))

cond: hypothyroidism, med: westhroid
KEYWORDS: ['gland', 'hormone', 'hypothyroidism', 'radiation', 'thyroid'] 

cond: hypothyroidism, med: levoxyl
KEYWORDS: ['gland', 'hormone', 'hypothyroidism', 'radiation', 'thyroid'] 

cond: hypothyroidism, med: cytomel
KEYWORDS: ['gland', 'hormone', 'hypothyroidism', 'radiation', 'thyroid'] 

cond: hyperthyroidism, med: tapazole
KEYWORDS: ['gland', 'hormone', 'hyperthyroidism', 'thyroid'] 

cond: hyperthyroidism, med: synthroid
KEYWORDS: ['gland', 'goiters', 'hormone', 'hypothyroidism', 'thyroid'] 

cond: hyperthyroidism, med: propylthiouracil
KEYWORDS: ['gland', 'hyperthyroidism', 'thyroid'] 

cond: hypotension, med: midodrine
KEYWORDS: ['pressure', 'hypotension'] 

cond: hypertension, med: lisinopril
KEYWORDS: ['pressure'] 

cond: hypertension, med: sectral
KEYWORDS: ['pressure'] 

cond: hypertension, med: atenolol
KEYWORDS: ['pressure', 'hypertension'] 

cond: hypertension, med: lotensin
KEYWORDS: ['pressure', 'hypertension'] 

cond: heart failu

In [13]:
# Should not have results
un_related = [
              ('GERD', 'synthroid'),
              ('hypertension', 'synthroid'),
              ('hypertension', 'insulin'),
              ('hypokalemia', 'insulin'),
              ('hyperlipidemia', 'nateglinide'),
              ('hyperkalemia', 'glipizide'),
              ('diabetes type 2', 'atorvastatin'),
              ('depression', 'cephalexin'),
              ('GERD', 'sectral'),
              ('diarrhea', 'bupropion'),
              ('cellulitis', 'lotensin'),
              ('heart failure', 'clomipramine'),
              ('depression', 'lisinopril'),
              ('GERD', 'furosemide'),
              ('cellulitis', 'aldactone'),
              ('diabetes', 'valsartan'),
              ('CHF', 'cephalexin'),
              ('hypokalemia', 'sulfamethoxazole'),
              ('hyperkalemia', 'clindamycin'),
              ('hypertension', 'lansoprazole'),
              ('hypotension', 'esomeprazole'),
              ('diabetes mellitus', 'aciphex'),
              ('COPD', 'tadalafil'),
              ('COPD', 'cephalexin'),
              ('erectile dysfunction', 'donepezil'),
              ('erectile dysfunction', 'wellbutrin'),
              ('urinary tract infection', 'donepezil'),
              ('UTI', 'tadalafil'),
              ('cancer', 'tadalafil'),
              ('cancer', 'donepezil'),
              ('cancer', 'procrit'),
              ('anemia', 'tadalafil'),
              ('anemia', 'cephalexin'),
              ('anemia', 'insulin'),
              ('dementia', 'cephalexin'),
              ('dementia', 'procrit'),
              ('dementia', 'insulin'),
              ('diabetes type 2', 'tadalafil'),
              ('diabetes type 2', 'abraxane'),
              ('hyperthyroidism', 'viagra'),
              ('hyperthyroidism', 'wellbutrin'),
              ('diarrhea', 'procrit'),
              ('diarrhea', 'tadalafil'),
              ('cellulitis', 'paraplatin'),
              ('cellulitis', 'exelon'),
              ('heart failure', 'aciphex'),
              ('heart failure', 'razadyne'),
              ('CHF', 'sildeenafil'),
              ('hyperkalemia', 'tadalafil'),
              ('hyperkalemia', 'razadyne'),
              ('hyperlipidemia', 'tadalafil'),
              ('hyperlipidemia', 'razadyne'),
              ('UTI', 'westhroid'),
              ('UTI', 'glipizide'),
              ('GERD', 'tadalafil'),
              ('GERD', 'insulin'),
              ('hypertension', 'tadalafil'),
              ('hypertension', 'insulin'),
              ('hypokalemia', 'procrit'),
              ('hypokalemia', 'exelon'), 

]

print_results(describe(un_related))

cond: GERD, med: synthroid
KEYWORDS: [] 

cond: hypertension, med: synthroid
KEYWORDS: [] 

cond: hypertension, med: insulin
KEYWORDS: ['stroke'] 

cond: hypokalemia, med: insulin
KEYWORDS: ['kidney'] 

cond: hyperlipidemia, med: nateglinide
KEYWORDS: ['stroke'] 

cond: hyperkalemia, med: glipizide
KEYWORDS: ['kidney'] 

cond: diabetes type 2, med: atorvastatin
KEYWORDS: [] 

cond: depression, med: cephalexin
KEYWORDS: [] 

cond: GERD, med: sectral
KEYWORDS: ['kidney'] 

cond: diarrhea, med: bupropion
KEYWORDS: [] 

cond: cellulitis, med: lotensin
KEYWORDS: [] 

cond: heart failure, med: clomipramine
KEYWORDS: [] 

cond: depression, med: lisinopril
KEYWORDS: [] 

cond: GERD, med: furosemide
KEYWORDS: ['fluid', 'kidney'] 

cond: cellulitis, med: aldactone
KEYWORDS: [] 

cond: diabetes, med: valsartan
KEYWORDS: ['kidney'] 

cond: CHF, med: cephalexin
KEYWORDS: [] 

cond: hypokalemia, med: sulfamethoxazole
KEYWORDS: [] 

cond: hyperkalemia, med: clindamycin
KEYWORDS: [] 

cond: hypertensi